In [14]:
import os
import streamlit as st
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chat_models import AzureChatOpenAI as AzureOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.agents import Tool, initialize_agent

In [15]:
load_dotenv()
 # Define OpenAI credentials
openai_api_type = os.getenv("OPENAI_API_TYPE")
openai_api_version = os.getenv("OPENAI_API_VERSION")
openai_api_base = os.getenv("OPENAI_API_BASE")
openai_api_key = os.getenv("OPENAI_API_KEY")
model_deployment_id = os.getenv("MODEL_DEPLOYMENT_ID")
# Set environment variables
if openai_api_base is not None:
    os.environ["OPENAI_API_BASE"] = openai_api_base
if openai_api_key is not None:
    os.environ["OPENAI_API_KEY"] = openai_api_key
if openai_api_type is not None:
    os.environ["OPENAI_API_TYPE"] = openai_api_type
if openai_api_version is not None:
     os.environ["OPENAI_API_VERSION"] = str(openai_api_version)

In [16]:
embeddings = OpenAIEmbeddings(
client=any,
deployment='embeddings',
model='text-embedding-ada-002',
openai_api_type="azure",
openai_api_base="https://copilot-openai.openai.azure.com",
openai_api_version="2023-05-15",
openai_api_key=os.environ["OPENAI_API_KEY"],
chunk_size=1
)

In [23]:
kb_sre = FAISS.load_local("livro_google", embeddings)
#llm = AzureOpenAI(model_kwargs={'engine':'trouble-buddy'}, client=any)
llm = AzureOpenAI(deployment_name='trouble-buddy', model_name='gpt-3.5-turbo', temperature=0.0, client=any)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=kb_sre.as_retriever(),
    chain_type='stuff',
    #memory=memory
)
tools = [
    Tool(
        name="Base de Conhecimento sobre o livro Building Secure and Reliable Systems do Google",
        func=qa.run,
        description=(
            'Use this tool when answering questions about SRE (Site Reliability Engineeing) and about best practices for building secure and reliable systems.'
        )
    )
]
agente = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_interactions=3,
    early_stopping_method='generate',
    memory=memory,
    handle_parsing_errors=True
)

In [24]:
agente("What is the importance of SRE?")



> Entering new AgentExecutor chain...
{
    "action": "Base de Conhecimento sobre o livro Building Secure and Reliable Systems do Google",
    "action_input": "What is the importance of SRE?"
}
Observation: SREs (Site Reliability Engineers) are important because they are responsible for ensuring that a company's systems and services are reliable, scalable, and efficient. They work to prevent outages and other issues that could negatively impact a company's users or customers. SREs also play a critical role in developing centralized infrastructure and organization-wide automation. Additionally, they work closely with security engineers to evaluate security and reliability risks and provide advice on how to mitigate those risks. Overall, SREs are essential for maintaining the stability and performance of a company's systems and services.
Thought:{
    "action": "Final Answer",
    "action_input": "SREs are responsible for ensuring that a company's systems and services are reliable, sca

{'input': 'What is the importance of SRE?',
 'chat_history': [HumanMessage(content='What is the importance of SRE?', additional_kwargs={}, example=False),
  AIMessage(content="SREs are responsible for ensuring that a company's systems and services are reliable, scalable, and efficient. They work to prevent outages and other issues that could negatively impact a company's users or customers. SREs also play a critical role in developing centralized infrastructure and organization-wide automation. Additionally, they work closely with security engineers to evaluate security and reliability risks and provide advice on how to mitigate those risks. Overall, SREs are essential for maintaining the stability and performance of a company's systems and services.", additional_kwargs={}, example=False)],
 'output': "SREs are responsible for ensuring that a company's systems and services are reliable, scalable, and efficient. They work to prevent outages and other issues that could negatively impact 